__Задача__

- Выгрузить данные о заказах из журнала обращения к api на просчет стоимости и времени
- Выделить из них признаки и целевую переменную: время производства, стоимость
- Очистить данные от персональной информации или коммерческой тайны
- Сохранить в файл для дальнейшего использования

__Выгрузка данных__

Часть заказов поступает из интернет-магазина. \
Оператор (менеджер, агент, продавец) составляет программируемые формы и описывает алгоритмы расчета цены и времени производства. \
То есть стоимость и время производства изделия для клиента расчитывается заранее установленой формулой. \
Формула возвращает лишь приблизительный результат и может расходиться с итогом. \
Особенно это касается времени производства

In [1]:
import re
import os
import json
import datetime
import pandas as pd
import numbers

from IPython.display import display
from IPython.core.display import HTML
from IPython.display import clear_output

import lib.utils as utils

- Прочитать файлы с журналом (строки с json)
- Распарсить запросы
- Объеденить запросы, совершенные в рамках одного просчета в одну запись

In [2]:
path_src = '../data/_logs_raw/'
files = os.listdir(path_src)

data_by_hash = dict()

def _hashf(log):
    _hashstr = []
    
    mov = utils.nget(log, ['req', 'body', 'argument', 'mov'])
    clid = utils.nget(log, ['req', 'body', 'clientId'])
    
    keys = list(mov)
    keys.sort()
    
    for k in keys:
        v = mov[k]
        if (isinstance(v, str) or isinstance(v, numbers.Number)):
            _hashstr.append(str(k) + '=' + str(v))
            
    _hashstr.append('clid=' + str(clid))
            
    return hash(';'.join(_hashstr))

# Прочитать каждый файл с журналом запросов в папке
for file in files:
    f = open(path_src + file, 'r', encoding='utf8')

    for log_str in f.readlines():
        # пропустить пустые строки
        if not log_str:
            continue;
            
        log = json.loads(log_str)

        # вернуть именование api метода
        method = utils.nget(log, ['req', 'query', 'method']) or utils.nget(log, ['req', 'body', 'method'])
        
        # вернуть состояние ответа
        state = log.get('state')
        
        # только запрос на калькуляцию цены и времени
        # только запросы с ответом
        if (not (
            method in ['calcProductDurations', 'calcProductPrice']
            and 'res-info' == state
        )):
            continue
            
        # код страницы
        post_id = utils.nget(log, ['req', 'body', 'argument', 'postId'])
        
        # вернуть параметры просчета (состояние формы)
        mov = utils.nget(log, ['req', 'body', 'argument', 'mov'])
 
        hashkey = _hashf(log)
    
        data_by_hash[hashkey] = _data = data_by_hash.get(hashkey, dict())
        
        _data.update(mov)
        _data['post_id'] = utils.nget(log, ['req', 'body', 'argument', 'postId'], 0)
        _data['gs'] = ''
        _data['gsid'] = ''
        _data['mmid'] = 0
        _data['user_groups'] = 0
        
        if 'calcProductDurations' == method:
            _data['prod_t'] = utils.nget(log, ['res', 'data', 'производство', 'duration'], 0)
            
        if 'calcProductPrice' == method:
            _data['sum'] = utils.nget(log, ['res', 'data', 'sumBeforeDiscount'], 0)
        
    # /for log_str in f.readlines():

# /for file in files:

In [3]:
# a = dict()

# for k in data_by_hash:
#     row = data_by_hash[k]
    
#     if row.get('post_id') == '22':
#             a[k] = (row.get('prod_t'), row.get('sum'))
        
# display(a, len(a))

Собрать окончательный вариант

Очистить полученные данные
- мусор
- персональные данные
- коммерческая тайна
- ошибки
- записи без целевой переменной

Признаки
- поля из пользовательской формы магазина
- код страницы

Целевые переменные
- prod_t -- время производства
- sum -- цена

Результат -- массив объектов с неопределенным составом полей
То есть строки могут содержать различные поля

Итоговые данные должны быть обезличены

In [9]:
_props_to_drop = {
    '0': 1,
    '__usergeo': 1,
    'загрузить макет': 1,
    'комментарий по штрих-коду': 1,
    'комментарий к заказу': 1,
    'комментарий': 1,
    '_шаблон_в_редакторе': 1,
    '_категория_в_редакторе': 1,
    'лицевая сторона': 1,
    'оборотная сторона': 1,
    'файлы': 1,
    'пункт выдачи': 1,
    'пожелания в дизайне': 1,
    'рабочее название': 1,
    'текст для бирки': 1,
    'файл для записи на магнитную ленту': 1,
    'tel': 1,
    'email': 1,
    'userId': 1,
    'template_id': 1,
}


def _is_valid_field(_dict, k):
    v = _dict[k]
    t = type(v)

    if _props_to_drop.get(k):
        return False
    
    # если поле не строка и не число -> пропустить
    if (not (isinstance(v, numbers.Number) or isinstance(v, str))):
        return False
    
    # поле, вероятно, содержит файлы -> пропустить
    if '[object FileList]' == v:
        return False

    if t == str:
        # поле, вероятно, содержит пути к файлам -> пропустить
        if os.path.isabs(v):
            return False

        # поле, вероятно, содержит пути к файлам (макропути) -> пропустить
        if utils.is_macro_path(v):
            return False
    
        # если поле содержит номера телефонов -> пропустить
        if utils.is_telephone_number(v):
            return False
    
        # если поле содержит email -> пропустить
        if utils.is_email(v):
            return False
    
        # поле, вероятно, ошибочное -> пропустить
        if len(re.findall(r'\.info', k)):
            return False
        
    return True


def _get_omit_fields(_dict):
    _omit = []
    
    for k in _dict:
        try:
            if not _is_valid_field(_dict, k):
                _omit.append(k)

        except BaseException as err:
            print(err, k, _dict[k])
            
    return _omit


_drop = []

for h in data_by_hash:
    _data = data_by_hash[h]

    if not _data.get('sum'):
        _drop.append(h)
        continue

    if not _data.get('prod_t'):
        _drop.append(h)
        continue
        
    utils.omit(_data, _get_omit_fields(_data), inplace=True)

utils.omit(data_by_hash, _drop, inplace=True)

1

1

In [5]:
# a = dict()

# for k in data_by_hash:
#     row = data_by_hash[k]
    
#     if row.get('post_id') == '22':
#             a[k] = (row.get('prod_t'), row.get('sum'))
        
# display(a, len(a))

{1889350343270374587: (583.0266293877551, 509),
 -16048856288749577: (574.8979591836735, 219),
 -653241084251274679: (576.8643661224489, 290),
 -5006402793615169506: (574.8979591836735, 219),
 -8452758447053012215: (575.9396957142857, 256),
 -4459867610310371176: (638.8795451082251, 421),
 -1674828474940301105: (575.9396957142857, 293),
 -8500761104655799733: (575.1241579591837, 227),
 1706118633306770153: (581.2720579591837, 447),
 7151648443785291171: (580.8956559183673, 433),
 386577976844397656: (540, 49169),
 6433087251973623649: (540, 57059),
 -6451176348737544939: (540, 57059),
 -3602403454588080024: (540, 50831),
 -6430834606498612348: (540, 45639),
 -3158949644563844637: (540, 22881),
 498962159726283879: (540, 22881),
 351219034276338436: (583.0266293877551, 509),
 -5336180199414386491: (583.0266293877551, 519),
 3232235126443783044: (583.0266293877551, 509),
 6958713443173864071: (583.0266293877551, 509),
 2944463834231737330: (583.0266293877551, 509),
 -27969111420518688: (

7365

In [10]:
final_data = list(data_by_hash.values())

In [11]:
df = pd.DataFrame(final_data)

In [12]:
df

,"длина,м","ширина,м",тираж,материал,цветность,качество печати,двойной проход печати,пропайка,люверсы,ламинация,...,вид изделия,добавить отверстие для крепления,форма отверстия,выбрать аксессуар,держатель таблички,макет первого верхнего поля,макет второго поля,макет третьего поля,макет четвертого поля,шаблон
0,1,1,1,ТСИЗШИ03,ТСПоЦВ02,ТСШр0К03,,Без пропайки,Без люверсов,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40191,NaN,NaN,10,ТЦБуМеГ2,ТСПоЦВ02,NaN,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40192,NaN,NaN,5,ТЦБуОф80,ТСПоЦВ02,NaN,NaN,NaN,NaN,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40193,NaN,NaN,20,ТЦБуКР01,NaN,NaN,NaN,NaN,NaN,ПЗРАЛАЛ2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40194,NaN,NaN,1000,ТЦБуКР01,ТСПоЦВ01,NaN,NaN,NaN,NaN,Односторонняя глянцевая,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# df[df['post_id'] == '22'][['mmid', 'prod_t','gs', 'gsid', 'sum']]

df[df['post_id'] == '22']['prod_t'].value_counts()

540    928
Name: prod_t, dtype: int64

In [8]:
# for row in data['movs']['rows']:
# #     if 'ГППОВиФо' == row['gs']:
#     if 1469845 == row['mmid']:
#         display(row)
#         break;

In [9]:
f = open('./data/logs/train.json', 'w', encoding='utf-8')
f.write(json.dumps(final_data, ensure_ascii=False))

16767676

Конечный вид -- массив javascript объектов

```javascript
// train.json
arr = [obj_1, obj_2, ... obj_n]
```

где
```javascript
obj = {
    mmid         : 0,           // идентификатор задачи/изделия/заказа (недоступен)
    gs           : 'ГПШрПЛ02',  // код номенклатуры-изделия (недоступен)
    gsid         : 'ТСИЗ3201',  // код номенклатуры-формы заказа (недоступен)
    prod_t       : 4462,        // время изготовления в минутах
    sum          : 2544,        // стоимость изделия
    user_groups  : 7,           // битовая-маска группы в которых состоит пользователь (недоступен)
    post_id      : 201,         // код страницы
    __источник   : 'ма',        // источник заказа (веб-магазин, внутренний заказ и проч.)

    // ... 
    // прочие поля
}
```